In [2]:
pip install num2words

     |████████████████████████████████| 102kB 2.4MB/s 


In [1]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [3]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Created on Sat May  9 22:34:44 2020

@author: sakib

"""


#change base directory
import os

base_path=r'/content'
working_directory = '/content'
data_folder = r'/content/data'
embedding_folder = r'/content/embeddings'
models_folder = r'/content/models'
model_directory = working_directory+'/'+'models'


def change_base_dir(base_dir_path):
    """ Change the working directopry of the code"""
    
    if not os.path.exists(base_dir_path):
        print ('creating directory', base_dir_path)
        os.makedirs(base_dir_path)
    print ('Changing base directory to ', base_dir_path)
    os.chdir(base_dir_path)
 
      

# base_folder='data'
# base_dir_path=base_path+'/'+base_folder
# change_base_dir(base_dir_path)
# download_data.extract_file('/media/sakib/alpha/work/EmotionDetectionDir/git/data/glove.6B.zip')



change_base_dir(base_path)
if not os.path.exists(data_folder):
        os.makedirs(data_folder)
if not os.path.exists(embedding_folder):
        os.makedirs(embedding_folder)
if not os.path.exists(models_folder):
        os.makedirs(models_folder)
  
#loading necessary files
from load_preprocess import load_data,load_data_embedding
from load_preprocess import preprocess_data
from load_preprocess import read_labels
import word2vec
import sswe
import glove_file
import designing_network
import download_data


#importing libraries
from keras.preprocessing.text import Tokenizer 
from keras.preprocessing.sequence import pad_sequences
from nltk.tokenize import word_tokenize
import pandas as pd



# Download data

print('Checking requirements:')
download_data.download_data(base_path = base_path)

#loading data
print('Step1: Loading Embedding training Dataset...')


dataset_embedding = load_data_embedding(working_directory+'/data/'+'training.1600000.processed.noemoticon.csv')
print('Step2: Shuffling data...')
dataset_embedding = dataset_embedding.sample(frac=1) # reshuffling the data
print('Step3: Preprocessing the texts...')
texts = preprocess_data(dataset_embedding)
labels = read_labels(dataset_embedding)


# building word2vec model
print('Step4: Building word2vec model...')
EMBEDDING_DIM = 100
w2v = word2vec.create_word2vec(directory = working_directory+'/'+'embeddings',texts = texts ,min_count = 1,EMBEDDING_DIM = EMBEDDING_DIM)



# loading twitter_dataset_small
print('Step6: Loading Twitter Dataset')
dataset = load_data(working_directory+'/'+'data'+'/'+'Tweets.csv')
texts = preprocess_data(dataset)
y = pd.get_dummies(dataset['Label']).values



#tokenizing
print ('Step7: Tokenizing...')

tokens = []
for line in texts:
    words = word_tokenize(line)
    tokens.append(words)



tokenizer_obj = Tokenizer()
tokenizer_obj.fit_on_texts(tokens)
sequences = tokenizer_obj.texts_to_sequences(tokens)


#padding
print ('Step8: Padding...')
tokenizer_word_index = tokenizer_obj.word_index
max_length = 150
review_pad = pad_sequences(sequences, maxlen = max_length)



# train test split
print('Step9: train and test set generation...')
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(review_pad, y, test_size = 0.20, random_state = 0)


# word2vec Embedding Matrix
print('Step10: Generating word2vec embedding matrix...')
num_words = len(tokenizer_word_index) + 1
embedding_matrix_w2v = word2vec.load_word2vec(working_directory+'/'+'embeddings'+'/'+'embeddings_w2v.txt', tokenizer_word_index=tokenizer_word_index, EMBEDDING_DIM=EMBEDDING_DIM)



Changing base directory to  /content


Using TensorFlow backend.


Checking requirements:
Extracting stanford data
Extracting glove data
Step1: Loading Embedding training Dataset...
Step2: Shuffling data...
Step3: Preprocessing the texts...
Step4: Building word2vec model...
Saving word2vec model in the disk


/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:253: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


Step6: Loading Twitter Dataset
Step7: Tokenizing...
Step8: Padding...
Step9: train and test set generation...
Step10: Generating word2vec embedding matrix...
Step11: designing lstm+w2v model...


NameError: ignored

In [13]:
import designing_networksssss
# training the word2vec model with lstm
print('Step11: designing lstm+w2v model...')

w2v_lstm = designing_networksssss.model_architecture_word2vec(embedding_matrix_w2v, num_words,EMBEDDING_DIM = EMBEDDING_DIM , max_length = max_length)
w2v_lstm, history = designing_networksssss.fit_network(w2v_lstm, X_train, X_test, y_train, y_test, epochs = 15)
designing_networksssss.save_network_model(w2v_lstm, modelname = 'w2v_lstm',directory = model_directory)
# loaded_model = designing_network.load_network_model( directory = working_directory+'/'+'models', jsonfile = 'w2v_lstm.json', h5file = 'w2v_lstm.h5')
# score = designing_network.analyze_performance(loaded_model, X_test, y_test)
# print("%s: %.2f%%" % (loaded_model.metrics_names[1], score[1]*100))


Step11: designing lstm+w2v model...
Model: "sequential_10"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_10 (Embedding)     (None, 150, 100)          1382300   
_________________________________________________________________
bidirectional_2 (Bidirection (None, 64)                34048     
_________________________________________________________________
dense_2 (Dense)              (None, 3)                 195       
Total params: 1,416,543
Trainable params: 34,243
Non-trainable params: 1,382,300
_________________________________________________________________
Train on 11712 samples, validate on 2928 samples
Epoch 1/15
11712/11712 [==============================] - 43s 4ms/step - loss: 0.4777 - accuracy: 0.7845 - val_loss: 0.3949 - val_accuracy: 0.8271
Epoch 2/15
11712/11712 [==============================] - 42s 4ms/step - loss: 0.3996 - accuracy: 0.8238 - val_loss: 0.3598 - val_accura